In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import utils

# 讀取訓練資料與測試資料

In [ ]:
train_data = pd.read_csv('new_train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data

# 讀取訓練圖片與測試圖片

In [ ]:
# 可以調整要壓縮的圖片像素
RESIZE = 40

In [ ]:
train_data_x = np.ndarray((len(train_data), RESIZE, RESIZE, 3))
train_data_y = np.ndarray((len(train_data), 14))
test_data_x = np.ndarray((len(test_data), RESIZE, RESIZE, 3))


In [ ]:
# 讀取圖片的function
# Reads an image from a file, decodes it into a dense tensor, and resizes it to a fixed shape.
def _parse_function(filename):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3) # Channels needed because some test images are b/w
    image_resized = tf.image.resize(image_decoded, [RESIZE, RESIZE])
    return image_resized

In [ ]:
# 讀取路徑後，將讀取的圖片調整大小，並存入train_data_x, train_data_y
for i, row in enumerate(train_data.values):
    image = _parse_function(row[14])
    train_data_x[i] = image
    train_data_y[i] = row[:14]
    if i%2000 == 0: print('Processed {} of {}'.format(i, len(train_data)))

In [ ]:
# # 讀取路徑後，將讀取的圖片調整大小，test_data_x
for i, row in enumerate(test_data.values):
    image = _parse_function(row[0])
    test_data_x[i] = image
    if i%2000 == 0: print('Processed {} of {}'.format(i, len(test_data)))

In [ ]:
# 畫出重新調整後的圖片
font = {'family': 'serif', 'color':  'white', 'weight': 'bold', 'size': 16}
plt.imshow(train_data_x[0]/255)
plt.scatter(train_data_y[0,3] * RESIZE, train_data_y[0,8] * RESIZE, 500, marker='x', color='red', linewidth=3)
plt.scatter(train_data_y[0,4] * RESIZE, train_data_y[0,9] * RESIZE, 500, marker='x', color='red', linewidth=3)
plt.text(3, 3, 'Predicted smile: {} '.format(train_data_y[0,11]),fontdict=font)

# 資料前處理
- 資料並不是讀進來就可以立即使用，有些參數需要前處理
- 例如，在本資料中`gender=1`為男性，`gender=2`為女性，如果要做binary classification需要轉換成`0`或`1`

# 建立模型
- 自行建立模型，並且以mutlitask任務的架構

# 匯出預測結果
- 利用pandas.Dataframe.to_csv()來匯出上傳到kaggle競賽
- 對於**binary classification**問題，欄位必須要有`Id`與`Category`
- 對於**regression**問題，欄位必須要有`Id`與`Expected`
- `Id`欄位就是測試資料的圖片位置例如:`AFLW/0001-image20056.jpg`

In [ ]:
# 將預測的結果放入，建立DataFrame物件，對於不同任務記得修改欄位名稱！！
tmp = pd.DataFrame({'Id':test_data['path'],'Category': None})

In [ ]:
tmp.to_csv("test_smile_result.csv", index=False)